<a href="https://colab.research.google.com/github/JonNData/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Jon_nguyen_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [x] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [X] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [3]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
import datetime

In [15]:
df['date'] = pd.to_datetime(df['created'])
# df['date'][49340]
start = datetime.datetime.strptime('2016-03-31', '%Y-%m-%d')
end = datetime.datetime.strptime('2016-06-01', '%Y-%m-%d')
start - end

datetime.timedelta(-62)

In [42]:
df_train = df[df['date']<end]
df_train.sort_values('date').head()
# X_train.sort_values('date').tail()
# looking at head and tail checks out

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,date,total_perks,description_length
5186,1.0,1,2016-04-01 22:12:41,X-LARGE Flex 1BR Loft! ~~ PRIME Greenwich Vill...,Astor Place,40.7302,-73.9924,3195,1 Astor Place,high,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-01 22:12:41,3,94.0
7945,1.0,0,2016-04-01 22:56:00,"This Enormous Studio Features: Harwood Floors,...",East 54th Street,40.7576,-73.9677,2000,230 East 54th Street,medium,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-01 22:56:00,4,357.0
6424,2.0,3,2016-04-01 22:57:15,--- East 31st St & Lexington Avenue --- This S...,East 31st St & Lexington Avenue,40.7388,-73.9851,5850,105 Lexington Avenue,high,1,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2016-04-01 22:57:15,9,411.0
7719,1.0,1,2016-04-01 23:26:07,Reduced Fee!! Priced To Rent!\rLarge Newly Upd...,West End Ave,40.7939,-73.9738,2745,700 West End Ave,medium,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2016-04-01 23:26:07,8,816.0
1723,1.0,1,2016-04-02 00:48:13,Phenomenal deal of the century!! This spacious...,E 88th street,40.7784,-73.9491,2400,401 E 88th street,medium,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-02 00:48:13,5,536.0


In [43]:
df_test = df[df['date']>end]
df_test.sort_values('date').tail()
# head and tail check. This is not ideal, but passing a two-sided inequality 
# did not work

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,date,total_perks,description_length
19943,1.0,2,2016-06-29 17:47:34,"Newly renovated, clean, quiet & very bright 2...",At Thompson St,40.7261,-74.0014,3045,132 Thompson St #20,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-29 17:47:34,0,208.0
16801,1.0,1,2016-06-29 17:56:12,Historic conversion in Greenpoint with awesome...,"100 Dupont St, Brooklyn, NY 11222",40.7358,-73.9560,2648,100 Dupont St,low,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2016-06-29 17:56:12,7,381.0
32633,1.0,1,2016-06-29 18:14:48,"Large, sunny one bedroom apartment with a sepa...",West 45th St. and 8th Ave.,40.7597,-73.9890,2650,305 West 45th St.,low,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-29 18:14:48,5,327.0
20560,1.0,3,2016-06-29 18:30:41,LOOK NO FURTHER!!\r\rMassive three bedroom apa...,williamsburg,40.7156,-73.9540,3200,32 Havemeyer Street,medium,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-29 18:30:41,2,393.0
17743,1.0,3,2016-06-29 21:41:47,LOOK NO FURTHER!!\r\rMassive three bedroom apa...,williamsburg,40.7156,-73.9540,3399,32 Havemeyer Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-29 21:41:47,0,393.0


In [59]:
# Feature engineering
df['total_perks'] = df['elevator'] + df['cats_allowed'] +df['common_outdoor_space'] + df['dining_room'] +df['dishwasher'] + df['dogs_allowed'] + df['exclusive'] +df['fitness_center'] + df['hardwood_floors'] +df['high_speed_internet'] +df['laundry_in_building'] +df['doorman'] + df['laundry_in_unit'] +df['loft'] +df['no_fee'] + df['outdoor_space'] + df['pre-war'] +df['roof_deck'] + df['swimming_pool'] + df['terrace'] +df['wheelchair_access'] + df['balcony'] +df['garden_patio']+df['new_construction']
# well I tried formatting that, and the notebook did not like that.

df['description_length'] = df['description'].str.len()
df['description_length'] = df['description_length'].fillna(0).astype(int)
df[['total_perks', 'description_length']]
# Niceeee

,total_perks,description_length
0,0,588
1,5,8
2,3,691
3,2,492
4,1,479
...,...,...
49347,5,787
49348,9,1125
49349,5,671
49350,5,735


In [52]:
df.isnull().sum()

bathrooms                  0
bedrooms                   0
created                    0
description             1425
display_address          133
latitude                   0
longitude                  0
price                      0
street_address            10
interest_level             0
elevator                   0
cats_allowed               0
hardwood_floors            0
dogs_allowed               0
doorman                    0
dishwasher                 0
no_fee                     0
laundry_in_building        0
fitness_center             0
pre-war                    0
laundry_in_unit            0
roof_deck                  0
outdoor_space              0
dining_room                0
high_speed_internet        0
balcony                    0
swimming_pool              0
new_construction           0
terrace                    0
exclusive                  0
loft                       0
garden_patio               0
wheelchair_access          0
common_outdoor_space       0
date          

In [60]:
df['description_length'].describe()

count    48817.000000
mean       601.806215
std        391.826042
min          0.000000
25%        341.000000
50%        564.000000
75%        809.000000
max       4466.000000
Name: description_length, dtype: float64

In [0]:
# Fit a multivariate linear regression
from sklearn.linear_model import LinearRegression
# Instantiate class
model = LinearRegression()

In [62]:
# define the variables of interest, in specific dfs
features = ['bedrooms', 'bathrooms', 'total_perks']
X_train = df_train[features]
X_test = df_test[features]
target = df_train['price']

# fit the model
model.fit(X_train, target)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [63]:
# predict
y_pred_train = model.predict(X_train)
y_pred_train

array([2840.37553641, 2762.20889276, 3859.99919689, ..., 2996.70882372,
       2604.72317446, 3076.02789836])

In [68]:
# Get the coefficients

bedcoeff, bathcoeff, totalpercoeff = model.coef_
print(f'The coefficient for bedrooms is {bedcoeff:.2f}')
print(f'The coefficient for bathrooms is {bathcoeff:.2f}')
print(f'The coefficient for total perks is {totalpercoeff:.2f}')
print('The intercept is', model.intercept_)

The coefficient for bedrooms is 391.99
The coefficient for bathrooms is 1936.28
The coefficient for total perks is 78.17
The intercept is 277.61314859139156


In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [74]:
# get the metrics from training set
mae_train = mean_absolute_error(df_train['price'], y_pred_train)
mse_train = mean_squared_error(target, y_pred_train)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(target, y_pred_train)

print('Root Mean Squared Error:', np.around(rmse_train))
print('Mean Absolute Error:', np.around(mae_train))
print('R^2:', r2_train)


Root Mean Squared Error: 1204.0
Mean Absolute Error: 795.0
R^2: 0.5331795878721651


In [75]:
# get the metrics from testing set

# Run the model on test set
y_pred_test = model.predict(X_test)
target_test = df_test['price']

# copy above and replace test variables
mae_test = mean_absolute_error(df_test['price'], y_pred_test)
mse_test = mean_squared_error(target_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(target_test, y_pred_test)

print('Root Mean Squared Error:', np.around(rmse_test))
print('Mean Absolute Error:', np.around(mae_test))
print('R^2:', r2_test)


Root Mean Squared Error: 1192.0
Mean Absolute Error: 800.0
R^2: 0.542772398486978


In [0]:
## Fiddle around to min MAE

def categorical_encode(df, col):
  """ Written by Maxie. Converts a categorical column into individual boolean ones.
  df (pd.Dataframe) - the dataframe to be operated on
  col (string) - the name of the categorical column you wish to encode. 
  """
  new_cols = df[col].value_counts().index

  for new_col in new_cols:
    df[new_col] = df[col] == new_col
    df[new_col] = df[new_col].apply(lambda x: 0 if x is False else 1)

  df = df.drop(columns=[col])

  return df

In [82]:
categorical_encode(df_train, 'interest_level')
categorical_encode(df_test, 'interest_level')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,date,total_perks,description_length,low,medium,high
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-24 07:54:24,0,588.0,0,1,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-12 12:19:27,5,8.0,1,0,0
11,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-03 03:21:22,0,690.0,1,0,0
14,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-01 03:11:01,3,569.0,1,0,0
24,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,2016-06-07 04:39:56,11,870.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49305,1.0,2,2016-06-16 04:20:46,Spacious sunny 2 bedroom apartment/ Queen size...,W 175 Street,40.8456,-73.9361,2295,575 W 175 Street,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-16 04:20:46,2,382.0,1,0,0
49310,1.0,3,2016-06-21 06:25:35,"Soaring 40 stories into the Manhattan skyline,...",Second Avenue,40.7817,-73.9497,3995,1751 Second Avenue,1,0,1,0,1,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2016-06-21 06:25:35,8,2289.0,0,0,1
49320,1.0,1,2016-06-02 13:24:18,Great deal for a one bedroom located in Prime ...,West 55th Street,40.7669,-73.9917,2727,448 West 55th Street,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-02 13:24:18,2,281.0,0,1,0
49332,1.0,2,2016-06-06 01:22:44,Don't miss out on this spacious and beautiful ...,West 98th Street,40.7957,-73.9705,4850,220 West 98th Street,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-06 01:22:44,4,519.0,1,0,0


In [83]:
# define the variables of interest, in specific dfs
features = ['bedrooms', 'bathrooms', 'total_perks', 'low', 'medium', 'high']
X_train = df_train[features]
X_test = df_test[features]
target = df_train['price']

# fit the model
model.fit(X_train, target)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [87]:
y_pred_new = model.predict(X_test)
y_pred_new

array([3935.1830962 , 3639.6443612 , 2789.19340557, ..., 2338.78388032,
       3554.54666509, 3019.03944373])

In [88]:
mae_new = mean_absolute_error(df_test['price'],y_pred_new)
mae_new
# Welp, a modest improvement for a modest effort.

756.6730928460714